# CS202 G3T2 Reading Assignment
In the following code, we intend to experiment and test different algorithms and possible solutions to the Travelling Salesman Problem (TSP).

## Table of contents
1. [Travelling Salesman Problem (TSP)](#the-travelling-salesman-problem-tsp)
1. [Algorithms](#list-of-algorithms)
1. [Experiment](#experimental-data)
1. [Results](#results)
1. [Conclusion](#conclusion)



In [122]:
# Import statements
import time
import random
from tabulate import tabulate
from functools import cmp_to_key

## The Travelling Salesman Problem (TSP)

Firstly, we define TSP as a problem involving an N x N adjacency matrix which represent the edge weights. For demonstration purposes, we chose to use a complete graph to represent it, with the assumptions that there are edges between every two nodes and that all edge weights are positive.

The following piece of code was taken and altered from the Wk12 exercises.

The function generates a random undirected graph with num_vertices number of "cities", with additional cost_min and cost_limit parameters to dictate the range of edge weights possible.

In [123]:
def generate_graph(
    num_vertices: int, cost_min: int, cost_limit: int
) -> list[list[int]]:
    
    graph: list[list[int]] = [
        [0 for _ in range(num_vertices)] for _ in range(num_vertices)
    ]

    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            weight: int = random.randint(cost_min, cost_limit)
            graph[i][j] = weight
            graph[j][i] = weight

    return graph

## Helper / utility functions

get_cost is a function used by several of the algorithms to obtain the cost of an entire path. It assumes that the path visits all nodes, while starting and ending at index 0.

apply_2_opt is a helper function for the local search functions which does 2-opt swapping by a pair of edges at x to x + 1 and y to y + 1.

In [124]:
# Must be of length N
# Will auto include the path back to start
def get_cost(graph: list[list[int]], visited: list[int]) -> int:
    n: int = len(visited)
    current_cost: int = 0

    for i in range(1, n):
        current_cost += graph[visited[i - 1]][visited[i]]

    current_cost += graph[visited[-1]][0]
    return current_cost


# x and y must be at least 2 apart
# If 1 apart, means adjacent, no diff
def apply_2_opt(visited: list[int], x: int, y: int) -> list[int]:
    new_path: list[int] = visited[: x + 1]
    new_path.extend(reversed(visited[x + 1 : y + 1]))
    new_path.extend(visited[y + 1 :])

    return new_path




# Generates a random path for the graph
def generate_random_path(graph: list[list[int]]) -> tuple[int, list[int]]:
    n: int = len(graph)
    path: list[int] = [0]
    cost: int = 0
    while len(path) != n:
        index: int = random.randint(1, n - 1)
        while index in path:
            index = random.randint(1, n - 1)
        cost += graph[path[-1]][index]
        path.append(index)
    cost += graph[path[-1]][index]
    return cost, path

## Perturbation operators

In [125]:
# Randomises vertices in a slice of the original path
def randomise_segments(path: list[int]) -> list[int]:

    n: int = len(path)

    chosen_vertices: list[int] = []

    # Get random segment index
    while True:
        for j in range(2):
            index: int = random.randint(1, n - 1)
            while index in chosen_vertices:
                index = random.randint(1, n - 1)
            chosen_vertices.append(index)

        chosen_vertices.sort()

        # NOTE: It is possible to add some criteria
        # to the chosen_vertices here, such as
        # setting a minimum length
        if chosen_vertices[1] - chosen_vertices[0] == 1:
            chosen_vertices.clear()
            # Randomise again
            continue

        break

    segment: list[int] = path[chosen_vertices[0] : chosen_vertices[1]]
    random.shuffle(segment)
    path[chosen_vertices[0] : chosen_vertices[1]] = segment

    return path


# Requires 4 distinct values
# Removes 4 edges, introduce 4 edges
def double_bridge_move(
    path: list[int],
) -> list[int]:

    n: int = len(path)

    vertices: list[int] = []
    while True:

        while len(vertices) != 4:
            index = random.randint(0, n - 1)
            while index in vertices:
                index = random.randint(0, n - 1)
            vertices.append(index)

        vertices.sort()
        continue_loop = False
        for i in range(1, 4):
            if vertices[i] - vertices[i - 1] == 1:
                continue_loop = True
                break
        if continue_loop:
            vertices.clear()
            continue

        break
    for i in range(4):
        vertices[i] += 1

    new_path: list[int] = path.copy()
    new_path[vertices[1] : vertices[2]], new_path[vertices[3] :] = (
        new_path[vertices[3] :],
        new_path[vertices[1] : vertices[2]],
    )

    return path

## List of algorithms
1. [Brute force](#brute-force-algorithm)
1. [Greedy](#greedy-algorithm)
1. [Local search](#local-search)
1. [Iterated local search (ILS) - Random segments](#iterated-local-search-ils---random-segments)
1. [Iterated local search (ILS) - Swap vertices](#iterated-local-search-ils---swap-vertices)
1. [Iterated local search (ILS) - Random initialisation](#iterated-local-search-ils---random-initialisation)

## Brute force algorithm

The most intuitive algorithm, is of course the brute force algorithm, which involves running through every path possible and then keep track of the path with the minimum cost. This guarantees that the optimal solution is found, but at the cost of exponential time, hence it is often not used.

In [126]:
def brute_force(graph: list[list[int]]) -> tuple[int, int, list[int]]:
    n: int = len(graph)
    optimal_path: list[int] = []
    iteration: int = 0
    min_cost: float = float("inf")

    # Recursive function to search all paths
    # path contains the index of all the vertices visited
    # cost is the current cost of the path
    def aux_func(path: list[int], cost: int) -> None:
        if len(path) == n:
            nonlocal iteration
            iteration += 1

            # Add the last path back to start vertex
            cost += graph[path[-1]][0]
            nonlocal min_cost

            if cost < min_cost:
                min_cost = cost
                nonlocal optimal_path
                optimal_path = path.copy()
            return

        for i in range(n):
            if i in path:
                continue

            # path[-1] is the last vertex visited
            # Add the path from last visited to current
            cost += graph[path[-1]][i]
            path.append(i)
            aux_func(path, cost)
            path.pop()
            cost -= graph[path[-1]][i]

    # Always start with the first vertex, with 0 cost
    aux_func([0], 0)
    return min_cost, iteration, optimal_path

## Greedy algorithm

The greedy algorithm does not guarantee an optimal solution. However, it makes up for it by offering a much faster speed. By picking the smallest edge weight for vertices not visited, it is able to iterate through the graph quickly, creating a single path. While not necessarily being the optimal solution, it offers a good initial solution as opposed to forming a completely random path. 

In [127]:
def greedy(graph: list[list[int]]) -> tuple[int, int, list[int]]:
    n: int = len(graph)
    iteration: int = 0

    # Stores the index of all visited vertices
    visited: list[int] = [0]
    cost: int = 0

    for i in range(1, n):
        min_cost = float("inf")
        min_idx = -1

        last_visited = visited[-1]
        for j in range(1, n):
            if j in visited:
                continue
            elif graph[last_visited][j] < min_cost:
                min_cost = graph[last_visited][j]
                min_idx = j

            iteration += 1

        visited.append(min_idx)
        cost += min_cost

    cost += graph[visited[-1]][0]
    return cost, iteration, visited

## Local search

Local search is used as an approximation algorithm as it too does not guarantee the optimal solution. This is due to its "locality" which does not explore possible "more costly" paths which may eventually lead to a better solution.

Local search requries an initial solution to be used, which is possible a form of optimisation for speed or results.

The following code presents a local search function which utilises the 2-opt helper functions to find a better solution.

In [128]:
def local_search_2_opt(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost

    iteration: int = 0

    # Terminates only after all edges have been
    # checked and no more better solution
    checked: int = 0
    while checked < num_edges:
        for i in range(n):
            for j in range(i + 1, n):
                if j - 1 == 1:
                    iteration += 1
                    checked += 1
                    continue

                new_path: list[int] = apply_2_opt(optimal_path, i, j)
                new_cost: list[int] = get_cost(graph, new_path)
                if new_cost < optimal_cost:
                    optimal_cost = new_cost
                    optimal_path = new_path
                    checked = 0
                else:
                    checked += 1
                iteration += 1

    return optimal_cost, iteration, optimal_path

## Local search with limit

The following is a special variation of 2-opt which has an iteration limit. Once it has reached the iteration limit, it will give up and return the current best. Utiilised by ILS further below.

In [129]:
def local_search_2_opt_limit(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    iteration_limit: int,
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost

    iteration: int = 0

    # Terminates only after all edges have been
    # checked and no more better solution
    checked: int = 0
    while checked < num_edges:
        for i in range(n):
            for j in range(i + 1, n):
                if j - 1 == 1:
                    iteration += 1
                    checked += 1
                    if iteration >= iteration_limit:
                        return optimal_cost, iteration, optimal_path
                    continue

                new_path: list[int] = apply_2_opt(optimal_path, i, j)
                new_cost: list[int] = get_cost(graph, new_path)
                if new_cost < optimal_cost:
                    optimal_cost = new_cost
                    optimal_path = new_path
                    checked = 0
                else:
                    checked += 1
                iteration += 1
                if iteration >= iteration_limit:
                    return optimal_cost, iteration, optimal_path

    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Double bridge move

Basing off the 2-opt local search, we can then create a solution to iterate between solutions generated by the local search. For this perturbation, double bridge move is chosen.

The logic of double bridge move is to remove 4 random edges, and introduce 4 new ones, forming a cross-like shape when bridging the gaps.

Local search is then run on the new paths to try and find better solutions.

Due to the nature of ILS, the termination criterion can also be changed to try and look for less or more solutions.

Below shows 3 variations of this ILS, with a Markovian, random walk, and restart acceptance criteria.

In [130]:
# ILS with 2-opt, double bridge, Markovian
def ILS_2_opt_double_bridge_markovian(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration

        path = double_bridge_move(path)

    return optimal_cost, iteration, optimal_path

In [131]:
# ILS with 2-opt, double bridge, random_walk
def ILS_2_opt_double_bridge_random_walk(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)
        optimal_cost = cost
        optimal_path = path
        iteration += ls_iteration

        path = double_bridge_move(path)

    return optimal_cost, iteration, optimal_path

In [132]:
# ILS with 2-opt, double bridge, restart
def ILS_2_opt_double_bridge_restart(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
    iteration_limit: int,
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt_limit(
            graph, cost, path, iteration_limit
        )
        optimal_cost = cost
        optimal_path = path
        iteration += ls_iteration
        if cost == iteration_limit:
            cost, path = generate_random_path(graph)
            continue

        path = double_bridge_move(path)

    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Random segments

A variation of ILS, with a different perturbation. This time, we randomly select segments in the graph and then randomly swap the vertices in these segments.

Below shows 3 variations of this ILS, with a Markovian, random walk, and restart acceptance criteria.

In [133]:
# Basing off 2-opt local search
# Randomly swap segments in the path
def ILS_2_opt_randomise_segment_markovian(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration

        path = randomise_segments(optimal_path)

    return optimal_cost, iteration, optimal_path

In [134]:
# ILS with 2-opt, randomise segments, restart
def ILS_2_opt_randomise_segment_restart(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
    iteration_limit: int,
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt_limit(
            graph, cost, path, iteration_limit
        )

        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration
        if cost == iteration_limit:
            cost, path = generate_random_path(graph)
            continue

        path = randomise_segments(path)

    return optimal_cost, iteration, optimal_path

In [135]:
# ILS using 2-opt, randomised segments, random walk
def ILS_2_opt_randomise_segment_random_walk(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:
    n: int = len(initial_solution)

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)

        optimal_cost = cost
        optimal_path = path
        iteration += ls_iteration

        path = randomise_segments(optimal_path)

    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Swap vertices

A variation of ILS, with a different perturbation. This time, we randomly swap vertices around, creating some disruptions which hopefully breaks the locality.

In [136]:
# Basing off 2-opt local search
# Randomly swap a few vertices
def ILS_2_opt_swap_vertices_markovian(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:
    n: int = len(initial_solution)

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration

        # Swap random pairs of vertices
        for i in range(random.randint(5, 10)):
            index1: int = random.randint(1, n - 1)
            index2: int = random.randint(1, n - 1)
            while index2 == index1:
                index2 = random.randint(1, n - 1)
            path[index1], path[index2] = path[index1], path[index2]

    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Random initialisation

Another variation of ILS, with a different perturbation. This time, instead of having an initial solution, we generate a random path in each iteration in hopes of finding one that local search may result in the best solution.

In [137]:
# Basing off 2-opt local search
# Randomly initalise paths
def ILS_2_opt_randomise_initial_markovian(
    graph: list[list[int]],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:
    initial_cost, initial_path = generate_random_path(graph)
    n: int = len(initial_path)
    tested_paths: list[list[int]] = [initial_path]

    optimal_path: list[int] = initial_path
    optimal_cost: int = initial_cost
    iteration: int = 0

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(
            graph, initial_cost, initial_path
        )
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration

        random_cost, random_path = generate_random_path(graph)
        while random_path in tested_paths:
            random_path = generate_random_path(graph)
        tested_paths.append(random_path)
        initial_path = random_path
        initial_cost = random_cost

    return optimal_cost, iteration, optimal_path

## Experimental Data

Based off the algorithms above, we decided to test the different algorithms with randomly generated "cities", as well as of different sizes. We decided to limit the number of vertices, N, at 200 as it becomes computationally expensive and time consuming to run.

Note: Brute force has been omitted for N > 10 as it simply runs for too long.

In [138]:
# Helper function for pretty printing
def print_data(data: list[tuple[str, int, int, list[int], float]]):
    print(
        tabulate(
            data,
            headers=[
                "Algorithm",
                "Cost",
                "Iterations",
                "Path",
                "Time taken",
            ],
            tablefmt="grid",
        )
    )
    print()


# Comparator function to sort by cost then iteration
def comparator_cost_iteration(algo1, algo2):
    if algo1[1] != algo2[1]:
        return algo1[1] - algo2[1]
    return algo1[2] - algo2[2]


def experiment(N: int, cost_min: int, cost_limit: int):
    print("Experiment starting with following parameters:")
    print("N: %d, cost_limit: %d" % (N, cost_limit))
    graph: list[list[int]] = generate_graph(N, cost_min, cost_limit)
    if N <= 20:
        print("Graph:")
        for arr in graph:
            print(arr)
        print()

    data: list[tuple[str, int, int, list[int], float]] = []

    if N <= 10:
        start_time: float = time.time()
        (cost, iteration, path) = brute_force(graph)
        elapsed_time: float = time.time() - start_time
        data.append(
            (
                "Brute Force",
                cost,
                iteration,
                path,
                elapsed_time,
            )
        )
        print("Brute Force done.")

    random_cost, random_path = generate_random_path(graph)
    random_cost: int = get_cost(graph, random_path)

    start_time: float = time.time()
    (greedy_cost, iteration, greedy_path) = greedy(
        graph
    )  # Use greedy solution calculated previously
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "Greedy",
            greedy_cost,
            iteration,
            greedy_path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("Greedy done.")

    start_time: float = time.time()
    (cost, iteration, path) = local_search_2_opt(
        graph, greedy_cost, greedy_path
    )  # Use greedy solution calculated previously
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt LS-Greedy",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt LS-Greedy done.")

    start_time: float = time.time()
    (cost, iteration, path) = local_search_2_opt(graph, greedy_cost, greedy_path)
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt LS-Random",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt LS-Random done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_segment_markovian(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy D_Bridge Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy D_Bridge Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_random_walk(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy D_Bridge Random_Walk",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy D_Bridge Random_Walk done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_restart(
        graph, greedy_cost, greedy_path, 10, 500
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy D_Bridge Restart",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy D_Bridge Restart done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_segment_markovian(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy R_Segment Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy R_Segment done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_swap_vertices_markovian(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy Swap_Vertices Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy Swap_Vertices Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_segment_markovian(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random R_Segment Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random R_Segment Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_segment_markovian(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random D_Bridge Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random D_Bridge Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_random_walk(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random D_Bridge Random_Walk",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random D_Bridge Random_Walk done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_restart(
        graph, random_cost, random_path, 10, 500
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random D_Bridge Restart",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random D_Bridge Restart done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_swap_vertices_markovian(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random Swap_Vertices Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random Swap_Vertices Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_initial_markovian(graph, 10)
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS R_Initial Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS R_Initial Markovian done.")

    print()

    # Sort by cost
    data.sort(key=cmp_to_key(comparator_cost_iteration))
    print("SORTED BY COST")
    print_data(data)
    print("WINNER:", data[0][0])
    print()

    # Sort by iteration
    data.sort(key=lambda x: x[2])
    print("SORTED BY ITERATIONS")
    print_data(data)
    print("WINNER:", data[0][0])
    print()


experiment(10, 1, 10)
experiment(20, 5, 20)
experiment(100, 5, 20)
experiment(200, 5, 30)
# experiment(500, 10, 50)

Experiment starting with following parameters:
N: 10, cost_limit: 10
Graph:
[0, 2, 8, 10, 1, 7, 2, 10, 7, 2]
[2, 0, 2, 2, 8, 6, 1, 9, 10, 5]
[8, 2, 0, 5, 1, 9, 9, 6, 3, 6]
[10, 2, 5, 0, 4, 7, 3, 5, 8, 8]
[1, 8, 1, 4, 0, 5, 2, 9, 10, 3]
[7, 6, 9, 7, 5, 0, 7, 8, 10, 9]
[2, 1, 9, 3, 2, 7, 0, 6, 6, 7]
[10, 9, 6, 5, 9, 8, 6, 0, 9, 4]
[7, 10, 3, 8, 10, 10, 6, 9, 0, 7]
[2, 5, 6, 8, 3, 9, 7, 4, 7, 0]

Brute Force done.
Greedy done.
2-opt LS-Greedy done.
2-opt LS-Random done.
2-opt ILS-Greedy D_Bridge Markovian done.
2-opt ILS-Greedy D_Bridge Random_Walk done.
2-opt ILS-Greedy D_Bridge Restart done.
2-opt ILS-Greedy R_Segment done.
2-opt ILS-Greedy Swap_Vertices Markovian done.
2-opt ILS-Random R_Segment Markovian done.
2-opt ILS-Random D_Bridge Markovian done.
2-opt ILS-Random D_Bridge Random_Walk done.
2-opt ILS-Random D_Bridge Restart done.
2-opt ILS-Random Swap_Vertices Markovian done.
2-opt ILS R_Initial Markovian done.

SORTED BY COST
+------------------------------------------+--------+-

2-opt LS-Greedy done.
2-opt LS-Random done.
2-opt ILS-Greedy D_Bridge Markovian done.
2-opt ILS-Greedy D_Bridge Random_Walk done.
2-opt ILS-Greedy D_Bridge Restart done.
2-opt ILS-Greedy R_Segment done.
2-opt ILS-Greedy Swap_Vertices Markovian done.
2-opt ILS-Random R_Segment Markovian done.
2-opt ILS-Random D_Bridge Markovian done.
2-opt ILS-Random D_Bridge Random_Walk done.
2-opt ILS-Random D_Bridge Restart done.
2-opt ILS-Random Swap_Vertices Markovian done.
2-opt ILS R_Initial Markovian done.

SORTED BY COST
+------------------------------------------+--------+--------------+--------+--------------+
| Algorithm                                |   Cost |   Iterations | Path   |   Time taken |
+==========================================+========+==============+========+==============+
| 2-opt LS-Greedy                          |    519 |        19800 | -      |    0.186175  |
+------------------------------------------+--------+--------------+--------+--------------+
| 2-opt LS-Random

## Results

As shown in the results obtained from the code above, we gather the following sentiments:

1. Greedy is always the fastest, but often does not have the optimal solution
1. There are a few contenders for what is the "best" solution but it is not clear as each has their drawbacks
1. For a decent approximation, **local search** with an initial **greedy** solution tends to perform well without too many iterations 

**Note that 2-opt ILS-Random D_Bridge Restart is hard capped at 5000 iterations due to iteration limit of 500 and 10 perbutations applied**

## Conclusion

The takeaway from this is that there is no "best" solution and there are simply too many factors specific to the problem that there is no generic good solution. Want a quick solution? Use a greedy algorithm. Want something with a bit more accuracy? Perhaps do a local search. If you need even more accurate results, then run iterative local search until you are happy with the result.